In [ ]:
import pump
import dask
# import ncar_jobqueue
import dask_jobqueue
import distributed

cluster = dask_jobqueue.SLURMCluster(
    cores=1, processes=1, memory='25GB',
    walltime='02:00:00', project='NCGD0043')

client = dask.distributed.Client(cluster)

In [ ]:
client

Client Scheduler: tcp://10.12.205.30:45969 Dashboard: https://jupyterhub.ucar.edu/dav/user/dcherian/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [ ]:
cluster.close(); client.close()

In [ ]:
cluster.adapt(minimum=2, maximum=72, wait_count=600)

In [ ]:
gcm1 = pump.model('../glade/TPOS_MITgcm_1_hb/HOLD/',
                  name='gcm1', full=True, budget=False)

/glade/u/home/dcherian/pump/pump/model/model.py:55: FutureWarning: In xarray version 0.13 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  self.dirname + '/obs_subset/annual-mean*.nc')
/gpfs/u/home/dcherian/python/xarray/xarray/backends/api.py:931: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).The datasets supplied require both concatenation and merging. From
xarray version 0.13 this will operation will require either using

Reading all files took 41.01607275009155 seconds


/glade/u/home/dcherian/pump/pump/model/model.py:257: FutureWarning: In xarray version 0.13 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  self.dirname + '/obs_subset/tao-*extract.nc')
/gpfs/u/home/dcherian/python/xarray/xarray/backends/api.py:931: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).The datasets supplied require both concatenation and merging. From
xarray version 0.13 this will operation will require either usin

In [ ]:
import numpy as np

def pv(ds):
    ds['b'] = ds.dens * -9.81/1025
    ds['b'].attrs['long_name'] = '$b$'
    ds['b'].attrs['description'] = 'buoyancy'

    f = 2*(np.pi/86400)  * np.sin(ds.latitude * np.pi/180)
    zeta = (ds.v.differentiate('longitude') - ds.u.differentiate('latitude'))/110e3
    q = ((f + zeta) * ds.b.differentiate('depth')
         - ds.v.differentiate('depth') * ds.b.differentiate('longitude')/110e3
         + ds.u.differentiate('depth') * ds.b.differentiate('latitude')/110e3)

    return q

In [ ]:
q = pv(gcm1.full[['u', 'v', 'dens']]
       .chunk({'latitude': 120, 'longitude': 500})
       .sel(latitude=slice(-3, 3), time=slice(None, None, 4)))

distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)


In [ ]:
q.data

,Array,Chunk
Bytes,183.07 GB,41.40 MB
Shape,"(120, 737, 345, 1500)","(60, 1, 345, 500)"
Count,658740 Tasks,4422 Chunks
Type,float32,numpy.ndarray


In [ ]:
q0 = q.sel(latitude=0, method='nearest').compute()

distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)


In [ ]:
len(q.__dask_graph__())

671919